In [1]:
import pandas as pd
import numpy as np
import plotly.express as px

In [2]:
orders = pd.read_csv("D:/github/Python/Python/Maven Pizza Challenge/Resources/pizza_sales/orders.csv", encoding='utf-8')                       
order_details = pd.read_csv("D:/github/Python/Python/Maven Pizza Challenge/Resources/pizza_sales/order_details.csv", encoding='utf-8')

In [3]:
orders_time = orders['time'].sort_index()                                                      
orders_count = order_details['order_id'].value_counts().sort_index()
orders_date = orders['date'].sort_index()

orders_datetime = (orders_date + ' ' + orders_time).astype('datetime64')                       

frame = {'Datetime': orders_datetime, 'Orders Count': orders_count}                            
total_orders = pd.DataFrame(frame).set_index('Datetime')

In [4]:
orders_per_month = total_orders.groupby(total_orders.index.month_name()).agg('count') + 1                     

bar_orders_per_month = px.bar(orders_per_month, y="Orders Count", x=orders_per_month.index,
            labels={
                "Orders Count": "No. of Orders",
                "Datetime": "Month"
            },
            title="Orders per month (2015)", width=800, height=400, 
            text=orders_per_month["Orders Count"], 
            color=orders_per_month["Orders Count"])

bar_orders_per_month.update_traces(textposition='auto')
bar_orders_per_month.update_xaxes(categoryorder='array', 
    categoryarray= ['January', 'February', 
    'March', 'April','May', 
    'June','July', 'August', 
    'September','October','November',
    'December',])

In [5]:
orders_per_day = total_orders.groupby(total_orders.index.day_name()).agg('count') + 1                     
bar_orders_per_day = px.bar(orders_per_day, y="Orders Count", x=orders_per_day.index,
            labels={
                "Orders Count": "No. of Orders",
                "Datetime": "Day"
            },
            title="Orders per day (2015)", width=800, height=400, 
            text=orders_per_day["Orders Count"], 
            color=orders_per_day["Orders Count"])

bar_orders_per_day.update_traces(textposition='inside')
bar_orders_per_day.update_xaxes(categoryorder='array', 
    categoryarray= ['Monday','Tuesday','Wednesday','Thursday', 'Friday', 'Saturday', 'Sunday'])

In [6]:
orders_per_hour = total_orders.resample("1h", label='right').count()                         
orders_per_hour = orders_per_hour.groupby(orders_per_hour.index.hour).agg('sum')

def get_orders_hour_morning(dataframe):
    morning = (dataframe >= 6) & (dataframe <= 11)
    return morning

def get_orders_hour_afternoon(dataframe):    
    afternoon = (dataframe >= 12) & (dataframe <= 17)
    return afternoon

def get_orders_hour_evening(dataframe):
    evening = (dataframe >= 18) & (dataframe <= 21)
    return evening

def get_orders_hour_night(dataframe):    
    night = (dataframe >= 22) | (dataframe <= 5)
    return night

def get_day_category(dataframe):
    day_categories = np.where(get_orders_hour_morning(dataframe) == True, "Morning", 
        np.where(get_orders_hour_afternoon(dataframe) == True, "Afternoon", 
            np.where(get_orders_hour_evening(dataframe) == True, "Evening", 
                np.where(get_orders_hour_night(dataframe) == True, "Night", np.NaN))))
    return day_categories

orders_per_hour['Category'] = get_day_category(orders_per_hour)

In [7]:
bar_orders_per_hour = px.bar(orders_per_hour, y="Orders Count", x=orders_per_hour.index, 
    color=orders_per_hour["Category"],
            labels={
                "Orders Count": "No. of Orders",
                "Datetime": "Hour",
                "Category": "Time of Day"
            },
            title="Orders per hour (2015)", width=800, height=400,
            hover_data={'Category'}, 
            text=orders_per_hour["Orders Count"])

bar_orders_per_hour.update_traces(textposition='auto')
bar_orders_per_hour.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [0, 5, 10, 15, 20],
        ticktext = ['12:00 a.m.', '5:00 a.m.', 
        '10:00 a.m.', '15:00 p.m.', '20:00 p.m.']
    )
)

In [9]:
orders_per_day.to_csv("D:/github/Python/Python/Maven Pizza Challenge/orders_per_day.csv", encoding='utf-8')
orders_per_hour.to_csv("D:/github/Python/Python/Maven Pizza Challenge/orders_per_hour.csv", encoding='utf-8')